![](img.png)
# LLMs in Real Products

Aim: Create an AI powered product to illustrate the general approach to building OpenAI "wrapper startups".

In this lecture, we'll work with the idea of developing a simple version of an application to tell a back and forth story, and create visualisations of it as it evolves.

Outline:
- Make a request to the OpenAI API to complete our text
- Using prompt engineering to pre-process the user's request
- Use the moderation API to ensure our responses aren't violating any guidelines
- Generate images to visualise our responses
- Wrap all of this in an API


Bonus:
- Extend the application to incorporate images

## Installing dependencies

In [4]:
!conda install -y -c conda-forge uvicorn
!conda install -c conda-forge openai
!conda install -c conda-forge fastapi


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



## Setting Up

In [10]:
import openai

openai.api_key = "YOUR_API_KEY"



## List the available models

Check the comparison in the documentation, [here](https://platform.openai.com/docs/models/gpt-3-5).

In [11]:
# openai.Model.list() # shows hundreds of lines

## Extra: Fine tune a model

If you acquire your own dataset, you can fine-tune a model to your specific domain.

Check out the details [here](https://platform.openai.com/docs/guides/fine-tuning).

## Making the request

Make sure to take a look into the different parameters and what they do using the documentation.

In [13]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a story!"}
    ]
)

print(completion)


{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Once upon a time, in a small village nestled in the heart of the forest, lived a young girl named Lily. Lily was kind and loving, and she spent most of her time helping others in the village. She would help the elderly with their chores and the younger children with their studies. Everyone loved Lily and looked up to her as a role model.\n\nOne day, while venturing deep into the forest to collect firewood, Lily stumbled upon a magical tree. The tree was unlike any she had ever seen before; it glowed with an otherworldly light and shimmered in the sunlight. Curious, Lily approached the tree, and as she reached out to touch it, she was suddenly transported to a mystical realm.\n\nIn this realm, Lily encountered all sorts of magical creatures, from fairies and unicorns to talking animals and enchanted plants. She explored this magical realm and met new friends. Everywhere she wen

## Unpacking the response

In [15]:
prediction = completion.choices[0].message["content"]
print(prediction)

Once upon a time, in a small village nestled in the heart of the forest, lived a young girl named Lily. Lily was kind and loving, and she spent most of her time helping others in the village. She would help the elderly with their chores and the younger children with their studies. Everyone loved Lily and looked up to her as a role model.

One day, while venturing deep into the forest to collect firewood, Lily stumbled upon a magical tree. The tree was unlike any she had ever seen before; it glowed with an otherworldly light and shimmered in the sunlight. Curious, Lily approached the tree, and as she reached out to touch it, she was suddenly transported to a mystical realm.

In this realm, Lily encountered all sorts of magical creatures, from fairies and unicorns to talking animals and enchanted plants. She explored this magical realm and met new friends. Everywhere she went, she spread joy and kindness, and all the creatures adored her.

But soon, Lily realized that her time in this re

### Wrap all of this into a function

In our case, these things are almost always going to need to happen together, so we should encapsulate them into the same function.

In [18]:
def get_completion(prompt):

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    prediction = completion.choices[0].message["content"]
    return prediction


get_completion("What is the meaning of life?")

'As an AI language model, I do not have the capability to provide a definitive answer to this philosophical question as it is open to interpretation and debate. Different people and cultures may hold various beliefs, values, and perspectives about the meaning of life. Some may find meaning in pursuing happiness, fulfilling personal goals and desires, helping others, contributing to society or a religious mission, seeking knowledge and truth, or embracing personal growth and self-discovery. Ultimately, the meaning of life is subjective and may vary from person to person.'

## Moderations

As a precaution against innapropriate messaging getting out to your users, you can run responses from the models through the moderations API.

In [19]:
response = get_completion("Tell me something innapropriate")
print(response)

result = openai.Moderation.create(input=response)
print(result)

As an AI language model, I am designed to be appropriate and respectful in all my responses. I am strictly programmed to refrain from generating any offensive or inappropriate content.
{
  "id": "modr-7Bv7HZ0W00BAvLkp8K4Up9vnYcFIp",
  "model": "text-moderation-004",
  "results": [
    {
      "categories": {
        "hate": false,
        "hate/threatening": false,
        "self-harm": false,
        "sexual": false,
        "sexual/minors": false,
        "violence": false,
        "violence/graphic": false
      },
      "category_scores": {
        "hate": 0.0001244684390258044,
        "hate/threatening": 1.1728182869319426e-09,
        "self-harm": 8.550870012058454e-10,
        "sexual": 6.172460416564718e-05,
        "sexual/minors": 3.983701191145883e-08,
        "violence": 7.368346814473625e-06,
        "violence/graphic": 3.6418961180118004e-09
      },
      "flagged": false
    }
  ]
}


## Define a function that takes in the chat so far and turns it into an image prompt

In [25]:
def create_image_prompt_from_text(text):

    text = """
    Distil the following text into a rich visual description of a picture from the scene in less than 400 characters.

    Describe the subject and surrounding scene first. Then go on to describe the angle and field of view of the camera, the style of the illustration, the lighting, and the mood.

    Story:
    """ + text

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": text}
        ]
    )
    response = response.choices[0].message["content"]
    return response

story = get_completion("Tell me a story!")
print("STORY")
print(story)
print("\n\n\n")
scene = create_image_prompt_from_text(story)
print("SCENE")
print(scene)


STORY
Once upon a time, there was a magical forest where animals of all kinds lived together harmoniously. The forest was so beautiful that people from all over the world would come to visit it. However, one day, a group of hunters stumbled upon the forest and began killing the animals for sport.

The animals were afraid and didn't know what to do. They tried to hide, but the hunters were too good at tracking them down. It seemed like all was lost, until the bravest deer in the forest devised a plan.

The deer rallied all of the animals together and told them about a magical flower that grew in a hidden corner of the forest. Legend had it that the flower had the power to make a wish come true if it was pure of heart.

So, the animals set out on a journey to find this magical flower. They traveled through the forest, braving all kinds of dangers along the way. Finally, they found the flower, and the bravest deer made a wish - a wish for the hunters to disappear forever.

Suddenly, the h

> # At this point, you should move away from Google Colab, and instead to VSCode then start working in Python files (`.py`)

## To serve our application as a product, we need to put all of this in a Python file, then turn it into an API

Put the utilities you've developed in this file into a Python file called `utils.py`.

## Building the back-end

The "back-end" of our application will be an API (application programmable interface). Here's the essentials of what you need to know about an API:
- Simply a program which runs on a computer somewhere, typically in a datacentre (on the cloud)
- Can accept "requests" and return responses
- Can implement different, custom responses to different requests

Our API will be able to respond to requests for things like:
- Text completion
- Moderation
- Converting our story so far into an image prompt
- Getting recommendations for follow-up prompts

_FastAPI_ is a Python library that can be used to create APIs. It defines simple ways to define the different things that your API can respond to. Read more about FastAPI [here](https://fastapi.tiangolo.com/).

Here's the basic structure:

In [ ]:
# highlight the basic strcuture for using fastapi

import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class EndpointData(BaseModel): # define the schema for the incoming data
    text: str # this schema expects an attribute of type string called "text"

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.post("/demo-endpoint") # define what happens to POST requests made to the /chat-response endpoint 
def predict(endpoint_data: EndpointData): # define a method that expects the incoming data to follow the Chat schema
    print("incoming payload text attribute:", endpoint_data.text)
    return {"response": "This is a response"} # return a response

if __name__ == '__main__': # if this file is being run directly
    uvicorn.run(app, host='localhost', port=8000) # run the API

# run the following from terminal to run this API: `uvicorn get_prediction:app --reload`

Before this works, you'll need to use `conda` to install some code that it depends on. You may want to do this in a new and separate conda environment.

In [ ]:
!conda install -y -c conda-forge uvicorn
!conda install -c conda-forge openai
!conda install -c conda-forge fastapi


Note again, this API code should be in a different `.py` file. To run it, run `uvicorn get_prediction: app --reload` from the terminal.

The API will continue to run until you kill the terminal process or until it runs into an error.

Remember that the API needs to be running still.

## Testing the back-end

We can test the back end by running it locally, and then making requests to it via Python (or however else you want to).


In [ ]:
import requests

response = requests.post("http://localhost:8000/demo-endpoint", json={"text": "This is a test"})
print(response)
print(response.json())

Now that the back end is working, you can begin to connect the front end to it. I've included a file called `index.html` that contains HTML which defines the structure of the webpage.

To run it locally with minimal hassle, click on the extensions tab on the far left of VSCode and download the extension shown in the following image.

![](./images/extension.png)

As per the instructions shown, you can hit `CTRL + SHIFT + L` when you're inside an HTML file, and it will run a preview of that webpage on a tab in your browser. It also updates as soon as you hit save.

When you run that you should see something like this:

![](./images/app.png)

If you take a look into the HTML file you can begin to decipher how the code works. You'll see it contains not just HTML, but also CSS and JavaScript.

But there's one secret weapon you can use here even if you don't know HTML, CSS, JavaScript... You can ask ChatGPT to write the code for you.

## Key Takeaways
- The OpenAI API is extremely easy to use
- Many products can be built using prompt engineering to adjust the input to OpenAI API endpoints to do things like create images and respond to a chat
- You can easily build a simple prototype of a product with just a few tools, in just a few hours
- You can use ChatGPT to write a mock up of the front-end with you